In [ ]:
# === CELL 1: SETUP ===
!pip install -q yfinance ta

import numpy as np
import pandas as pd
import yfinance as yf
import json
import time
import random
from datetime import datetime, timedelta
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

print("🔮 Forecast Evolution Optimizer Ready")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# === CELL 2: EVOLUTION CONFIG ===

EVOLUTION_CONFIG = {
    'population_size': 60,      # 60 forecast strategies compete
    'generations': 40,          # 40 generations of evolution
    'mutation_rate': 0.35,      # 35% chance of mutation
    'crossover_rate': 0.7,      # 70% chance of breeding
    'elite_keep': 6,            # Keep top 6 unchanged
    'tournament_size': 5,       # Tournament selection
}

# === FORECASTER PARAMETERS TO EVOLVE ===
# Each parameter has: [min, max, current_best]
FORECAST_PARAMS = {
    # === INDICATOR WEIGHTS (how much each indicator matters) ===
    'w_rsi': [0.0, 2.0, 1.0],              # RSI weight
    'w_macd': [0.0, 2.0, 1.0],             # MACD weight
    'w_ema_trend': [0.0, 2.0, 1.0],        # EMA ribbon weight
    'w_volume': [0.0, 2.0, 0.5],           # Volume weight
    'w_momentum': [0.0, 2.0, 1.0],         # Momentum weight
    'w_volatility': [0.0, 2.0, 0.5],       # Volatility (ATR) weight
    
    # === THRESHOLD PARAMETERS ===
    'rsi_oversold': [15, 40, 30],          # RSI buy threshold
    'rsi_overbought': [60, 85, 70],        # RSI sell threshold
    'rsi_neutral_low': [35, 50, 40],       # RSI neutral zone lower
    'rsi_neutral_high': [50, 65, 60],      # RSI neutral zone upper
    
    # === CONFIDENCE PARAMETERS ===
    'base_confidence': [0.4, 0.7, 0.55],   # Starting confidence
    'confidence_boost': [0.05, 0.25, 0.15], # Boost for strong signals
    'confidence_decay': [0.85, 0.98, 0.92], # Daily confidence decay
    
    # === FORECAST MECHANICS ===
    'drift_bull': [0.001, 0.005, 0.002],   # Daily bullish drift
    'drift_bear': [-0.005, -0.001, -0.002], # Daily bearish drift
    'volatility_scale': [0.3, 1.0, 0.5],   # How much volatility affects forecast
    'decay_start_day': [5, 15, 10],        # When signal starts decaying
    'max_daily_move': [0.03, 0.10, 0.05],  # Max daily price change
    
    # === MULTI-TIMEFRAME WEIGHTS ===
    'ema_short_period': [5, 13, 8],        # Short EMA period
    'ema_long_period': [21, 55, 34],       # Long EMA period
    'momentum_lookback': [3, 10, 5],       # Days for momentum calc
    
    # === SIGNAL THRESHOLDS ===
    'signal_buy_threshold': [0.3, 0.7, 0.5],   # Min score to trigger BUY
    'signal_sell_threshold': [-0.7, -0.3, -0.5], # Max score to trigger SELL
}

WATCHLIST = [
    'APLD', 'SERV', 'MRVL', 'NVDA', 'AMD', 'MU', 'CRDO',
    'SMR', 'OKLO', 'LEU', 'UUUU', 'CCJ',
    'HOOD', 'LUNR', 'SNOW',
    'IONQ', 'RGTI', 'QUBT',
    'TSLA', 'META', 'GOOGL',
    'SPY', 'QQQ',
    'RIVN', 'LYFT'
]

print("🔮 FORECAST EVOLUTION CONFIG")
print(f"   Population: {EVOLUTION_CONFIG['population_size']}")
print(f"   Generations: {EVOLUTION_CONFIG['generations']}")
print(f"   Evolvable params: {len(FORECAST_PARAMS)}")
print(f"   Tickers: {len(WATCHLIST)}")

In [ ]:
# === CELL 3: LOAD DATA ===
print("📥 Loading data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

# Split 70/30 train/test
train_data = {}
test_data = {}
for ticker, df in data_dict.items():
    split = int(len(df) * 0.7)
    train_data[ticker] = df.iloc[:split].reset_index(drop=True)
    test_data[ticker] = df.iloc[split:].reset_index(drop=True)

print(f"\n✅ Train: {len(train_data[list(train_data.keys())[0]])} days")
print(f"✅ Test: {len(test_data[list(test_data.keys())[0]])} days")

In [ ]:
# === CELL 4: FEATURE ENGINE FOR FORECASTING ===
def compute_forecast_features(df, params=None):
    """Compute features using evolved parameters"""
    if params is None:
        params = {name: best for name, (_, _, best) in FORECAST_PARAMS.items()}
    
    df = df.copy()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    
    # === RSI (evolved period could be added) ===
    delta = c.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    df['rsi'] = 100 - (100 / (1 + gain / (loss + 1e-10)))
    
    # === EMAs (evolved periods) ===
    short_ema = int(params.get('ema_short_period', 8))
    long_ema = int(params.get('ema_long_period', 34))
    df['ema_short'] = c.ewm(span=short_ema).mean()
    df['ema_long'] = c.ewm(span=long_ema).mean()
    
    # Standard EMAs for ribbon
    for p in [8, 13, 21, 34, 55]:
        df[f'ema_{p}'] = c.ewm(span=p).mean()
    
    # EMA trend signal
    df['ema_bullish'] = (df['ema_short'] > df['ema_long']).astype(float)
    df['ema_spread'] = (df['ema_short'] - df['ema_long']) / c * 100
    
    # === MACD ===
    df['macd'] = c.ewm(span=12).mean() - c.ewm(span=26).mean()
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_bullish'] = (df['macd'] > df['macd_signal']).astype(float)
    
    # === Momentum (evolved lookback) ===
    mom_days = int(params.get('momentum_lookback', 5))
    df['momentum'] = c.pct_change(mom_days) * 100
    df['momentum_5d'] = c.pct_change(5) * 100
    df['momentum_10d'] = c.pct_change(10) * 100
    
    # === Volatility (ATR-based) ===
    tr1 = h - l
    tr2 = abs(h - c.shift(1))
    tr3 = abs(l - c.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df['atr'] = tr.rolling(14).mean()
    df['atr_pct'] = df['atr'] / c * 100
    df['volatility'] = c.pct_change().rolling(20).std() * np.sqrt(252) * 100
    
    # === Volume ===
    df['vol_sma'] = v.rolling(20).mean()
    df['vol_ratio'] = v / (df['vol_sma'] + 1)
    df['vol_spike'] = (df['vol_ratio'] > 2).astype(float)
    
    # === Price position ===
    df['high_20d'] = h.rolling(20).max()
    df['low_20d'] = l.rolling(20).min()
    df['price_position'] = (c - df['low_20d']) / (df['high_20d'] - df['low_20d'] + 1e-10)
    
    # === Trend strength ===
    df['trend_5d'] = np.sign(c.diff(5))
    df['trend_10d'] = np.sign(c.diff(10))
    df['trend_21d'] = np.sign(c.diff(21))
    df['trend_align'] = (df['trend_5d'] + df['trend_10d'] + df['trend_21d']) / 3
    
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill().fillna(0)
    return df

print("🧠 Computing features...")
train_features = {t: compute_forecast_features(df) for t, df in train_data.items()}
test_features = {t: compute_forecast_features(df) for t, df in test_data.items()}
print("✅ Features ready")

In [ ]:
# === CELL 5: FORECAST DNA CLASS ===

class ForecastDNA:
    """A forecasting strategy's genetic code"""
    
    def __init__(self, params=None):
        if params:
            self.params = params
        else:
            # Random initialization
            self.params = {}
            for name, (min_v, max_v, _) in FORECAST_PARAMS.items():
                if isinstance(min_v, int) and isinstance(max_v, int):
                    self.params[name] = random.randint(min_v, max_v)
                else:
                    self.params[name] = random.uniform(min_v, max_v)
        
        self.fitness = 0
        self.accuracy = 0
        self.direction_accuracy = 0
    
    def mutate(self):
        """Randomly mutate one parameter"""
        param = random.choice(list(self.params.keys()))
        min_v, max_v, _ = FORECAST_PARAMS[param]
        
        # Mutation: adjust by 10-30%
        current = self.params[param]
        range_size = max_v - min_v
        mutation = random.uniform(-0.3, 0.3) * range_size
        new_val = current + mutation
        
        # Clamp to bounds
        new_val = max(min_v, min(max_v, new_val))
        
        if isinstance(min_v, int) and isinstance(max_v, int):
            new_val = int(new_val)
        
        self.params[param] = new_val
    
    @staticmethod
    def crossover(parent1, parent2):
        """Breed two forecasters"""
        child_params = {}
        for param in parent1.params:
            if random.random() < 0.5:
                child_params[param] = parent1.params[param]
            else:
                child_params[param] = parent2.params[param]
            
            # 20% chance to blend
            if random.random() < 0.2:
                blend = (parent1.params[param] + parent2.params[param]) / 2
                min_v, max_v, _ = FORECAST_PARAMS[param]
                if isinstance(min_v, int) and isinstance(max_v, int):
                    blend = int(blend)
                child_params[param] = blend
        
        return ForecastDNA(child_params)
    
    def __repr__(self):
        return f"ForecastDNA(fitness={self.fitness:.1f}, acc={self.accuracy:.1f}%)"

print("✅ ForecastDNA class ready")
# Test
test_dna = ForecastDNA()
print(f"   Sample DNA params: {len(test_dna.params)} genes")

In [ ]:
# === CELL 6: EVOLVED FORECASTER ===

class EvolvedForecaster:
    """Forecaster using evolved parameters"""
    
    def __init__(self, dna):
        self.p = dna.params
    
    def compute_signal_score(self, row):
        """
        Compute weighted signal score from indicators.
        Returns: score from -1 (very bearish) to +1 (very bullish)
        """
        score = 0
        total_weight = 0
        
        # === RSI Signal ===
        rsi = row.get('rsi', 50)
        rsi_signal = 0
        if rsi < self.p['rsi_oversold']:
            rsi_signal = 1.0  # Oversold = bullish
        elif rsi > self.p['rsi_overbought']:
            rsi_signal = -1.0  # Overbought = bearish
        elif rsi < self.p['rsi_neutral_low']:
            rsi_signal = 0.3
        elif rsi > self.p['rsi_neutral_high']:
            rsi_signal = -0.3
        
        score += rsi_signal * self.p['w_rsi']
        total_weight += self.p['w_rsi']
        
        # === MACD Signal ===
        macd_bullish = row.get('macd_bullish', 0.5)
        macd_hist = row.get('macd_hist', 0)
        macd_signal = (macd_bullish - 0.5) * 2  # -1 to 1
        if macd_hist > 0:
            macd_signal = min(1.0, macd_signal + 0.3)
        elif macd_hist < 0:
            macd_signal = max(-1.0, macd_signal - 0.3)
        
        score += macd_signal * self.p['w_macd']
        total_weight += self.p['w_macd']
        
        # === EMA Trend Signal ===
        ema_bullish = row.get('ema_bullish', 0.5)
        ema_spread = row.get('ema_spread', 0)
        ema_signal = (ema_bullish - 0.5) * 2
        # Amplify if spread is large
        if abs(ema_spread) > 2:
            ema_signal *= 1.5
        ema_signal = np.clip(ema_signal, -1, 1)
        
        score += ema_signal * self.p['w_ema_trend']
        total_weight += self.p['w_ema_trend']
        
        # === Volume Signal ===
        vol_ratio = row.get('vol_ratio', 1)
        momentum = row.get('momentum', 0)
        # High volume confirms direction
        if vol_ratio > 1.5:
            vol_signal = np.sign(momentum) * min(1.0, (vol_ratio - 1) / 2)
        else:
            vol_signal = 0
        
        score += vol_signal * self.p['w_volume']
        total_weight += self.p['w_volume']
        
        # === Momentum Signal ===
        mom_signal = np.clip(momentum / 10, -1, 1)  # Normalize
        
        score += mom_signal * self.p['w_momentum']
        total_weight += self.p['w_momentum']
        
        # === Volatility Signal ===
        atr_pct = row.get('atr_pct', 2)
        # High volatility = uncertainty, reduce confidence
        vol_penalty = min(1.0, atr_pct / 5) * 0.3
        
        # Normalize score
        if total_weight > 0:
            score = score / total_weight
        
        # Apply volatility dampening
        score *= (1 - vol_penalty * self.p['w_volatility'])
        
        return np.clip(score, -1, 1)
    
    def generate_signal(self, row):
        """
        Generate BUY/SELL/HOLD signal with confidence.
        Returns: (signal, confidence)
        """
        score = self.compute_signal_score(row)
        
        if score > self.p['signal_buy_threshold']:
            signal = 'BUY'
            confidence = self.p['base_confidence'] + self.p['confidence_boost'] * abs(score)
        elif score < self.p['signal_sell_threshold']:
            signal = 'SELL'
            confidence = self.p['base_confidence'] + self.p['confidence_boost'] * abs(score)
        else:
            signal = 'HOLD'
            confidence = self.p['base_confidence']
        
        return signal, min(0.95, confidence), score
    
    def forecast_price(self, current_price, signal, confidence, day, volatility):
        """
        Generate price forecast for a specific day.
        """
        # Decay confidence over time
        decay_factor = 1.0
        if day > self.p['decay_start_day']:
            decay_factor = self.p['confidence_decay'] ** (day - self.p['decay_start_day'])
        
        effective_conf = confidence * decay_factor
        
        # Determine drift based on signal
        if signal == 'BUY':
            drift = self.p['drift_bull'] * effective_conf
        elif signal == 'SELL':
            drift = self.p['drift_bear'] * effective_conf
        else:
            drift = 0
        
        # Add volatility component
        vol_scale = self.p['volatility_scale']
        random_component = np.random.normal(0, volatility * vol_scale / 100)
        
        # Calculate price change
        price_change = drift + random_component
        
        # Clamp to max daily move
        price_change = np.clip(price_change, -self.p['max_daily_move'], self.p['max_daily_move'])
        
        new_price = current_price * (1 + price_change)
        
        return new_price, effective_conf

print("✅ EvolvedForecaster class ready")

In [ ]:
# === CELL 7: FITNESS EVALUATOR ===

def evaluate_forecast_fitness(dna, features_dict, data_dict, forecast_days=7):
    """
    Evaluate forecaster accuracy.
    
    Fitness based on:
    - Direction accuracy (did we predict up/down correctly?)
    - Magnitude accuracy (how close was our forecast?)
    - Signal quality (did BUY signals lead to gains?)
    """
    forecaster = EvolvedForecaster(dna)
    
    direction_correct = 0
    direction_total = 0
    magnitude_errors = []
    signal_returns = {'BUY': [], 'SELL': [], 'HOLD': []}
    
    tickers = list(features_dict.keys())
    
    for ticker in tickers:
        df = features_dict[ticker]
        data = data_dict[ticker]
        
        # Test on multiple points
        test_points = range(60, len(df) - forecast_days - 1, 5)  # Every 5 days
        
        for start_idx in test_points:
            try:
                row = df.iloc[start_idx].to_dict()
                current_price = float(data['Close'].iloc[start_idx])
                
                # Generate signal
                signal, confidence, score = forecaster.generate_signal(row)
                
                # Get actual future prices
                future_prices = []
                for d in range(1, forecast_days + 1):
                    if start_idx + d < len(data):
                        future_prices.append(float(data['Close'].iloc[start_idx + d]))
                
                if len(future_prices) < forecast_days:
                    continue
                
                # Actual return over forecast period
                actual_return = (future_prices[-1] / current_price - 1) * 100
                
                # Direction accuracy
                predicted_direction = 1 if signal == 'BUY' else (-1 if signal == 'SELL' else 0)
                actual_direction = 1 if actual_return > 1 else (-1 if actual_return < -1 else 0)
                
                if predicted_direction != 0:  # Only count non-HOLD predictions
                    direction_total += 1
                    if predicted_direction == actual_direction:
                        direction_correct += 1
                
                # Signal quality (did signal lead to profit?)
                signal_returns[signal].append(actual_return)
                
                # Generate forecast and measure error
                volatility = float(row.get('atr_pct', 2))
                forecasted_price = current_price
                
                for d in range(forecast_days):
                    forecasted_price, _ = forecaster.forecast_price(
                        forecasted_price, signal, confidence, d + 1, volatility
                    )
                
                forecast_return = (forecasted_price / current_price - 1) * 100
                magnitude_error = abs(forecast_return - actual_return)
                magnitude_errors.append(magnitude_error)
                
            except Exception as e:
                continue
    
    # Calculate metrics
    direction_accuracy = (direction_correct / max(direction_total, 1)) * 100
    avg_magnitude_error = np.mean(magnitude_errors) if magnitude_errors else 100
    
    # Signal quality scores
    buy_avg = np.mean(signal_returns['BUY']) if signal_returns['BUY'] else 0
    sell_avg = np.mean(signal_returns['SELL']) if signal_returns['SELL'] else 0
    
    # Good BUY signals should have positive returns
    # Good SELL signals should have negative returns (or we avoided losses)
    buy_quality = max(0, buy_avg)  # Reward positive returns on BUY
    sell_quality = max(0, -sell_avg)  # Reward avoided losses on SELL
    signal_quality = buy_quality + sell_quality
    
    # FITNESS FORMULA:
    # High direction accuracy + low magnitude error + good signal quality
    fitness = (
        direction_accuracy * 2 +              # Weight direction accuracy heavily
        max(0, 50 - avg_magnitude_error) +    # Penalty for large errors
        signal_quality * 10                    # Reward good signals
    )
    
    return fitness, direction_accuracy, avg_magnitude_error, buy_avg, sell_avg

print("✅ Fitness evaluator ready")

In [ ]:
# === CELL 8: EVOLUTION ENGINE ===

def evolve_forecasters(population, features, data):
    """One generation of evolution"""
    
    # Evaluate all
    for dna in population:
        dna.fitness, dna.accuracy, _, _, _ = evaluate_forecast_fitness(dna, features, data)
    
    # Sort by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    # Keep elite
    new_pop = population[:EVOLUTION_CONFIG['elite_keep']]
    
    # Fill rest with offspring
    while len(new_pop) < EVOLUTION_CONFIG['population_size']:
        # Tournament selection
        def tournament():
            contestants = random.sample(population[:25], EVOLUTION_CONFIG['tournament_size'])
            return max(contestants, key=lambda x: x.fitness)
        
        parent1 = tournament()
        parent2 = tournament()
        
        # Crossover
        if random.random() < EVOLUTION_CONFIG['crossover_rate']:
            child = ForecastDNA.crossover(parent1, parent2)
        else:
            child = ForecastDNA(deepcopy(parent1.params))
        
        # Mutation
        if random.random() < EVOLUTION_CONFIG['mutation_rate']:
            child.mutate()
        
        new_pop.append(child)
    
    return new_pop

print("✅ Evolution engine ready")

In [ ]:
# === CELL 9: RUN EVOLUTION ===
print("=" * 70)
print("🔮 STARTING FORECAST EVOLUTION")
print("=" * 70)

# Initialize population
population = [ForecastDNA() for _ in range(EVOLUTION_CONFIG['population_size'])]

# Add human-tuned baseline
human_params = {name: best for name, (_, _, best) in FORECAST_PARAMS.items()}
population[0] = ForecastDNA(human_params)

best_ever = None
history = []

start_time = time.time()

for gen in range(EVOLUTION_CONFIG['generations']):
    # Evolve on TRAINING data only
    population = evolve_forecasters(population, train_features, train_data)
    
    best = population[0]
    avg_fitness = np.mean([d.fitness for d in population[:10]])
    avg_acc = np.mean([d.accuracy for d in population[:10]])
    
    # Track best ever
    if best_ever is None or best.fitness > best_ever.fitness:
        best_ever = ForecastDNA(deepcopy(best.params))
        best_ever.fitness = best.fitness
        best_ever.accuracy = best.accuracy
    
    history.append({
        'gen': gen,
        'best_fitness': best.fitness,
        'best_accuracy': best.accuracy,
        'avg_fitness': avg_fitness
    })
    
    if gen % 5 == 0:
        elapsed = time.time() - start_time
        print(f"Gen {gen:3d} | Fit: {best.fitness:8.1f} | Dir Acc: {best.accuracy:5.1f}% | Avg: {avg_acc:5.1f}% | {elapsed:.0f}s")

print("-" * 70)
print(f"\n🏆 EVOLUTION COMPLETE in {time.time()-start_time:.0f}s")
print(f"   Best Fitness: {best_ever.fitness:.1f}")
print(f"   Best Direction Accuracy: {best_ever.accuracy:.1f}%")

In [ ]:
# === CELL 10: TEST ON UNSEEN DATA ===
print("=" * 70)
print("🧪 TESTING EVOLVED FORECASTER ON UNSEEN DATA")
print("=" * 70)

# Test evolved best
evolved_fit, evolved_acc, evolved_err, evolved_buy, evolved_sell = evaluate_forecast_fitness(
    best_ever, test_features, test_data
)

# Test human baseline
human_dna = ForecastDNA(human_params)
human_fit, human_acc, human_err, human_buy, human_sell = evaluate_forecast_fitness(
    human_dna, test_features, test_data
)

print(f"\n{'Metric':<25} {'Human':>15} {'Evolved':>15} {'Winner':>10}")
print("-" * 65)
print(f"{'Direction Accuracy':<25} {human_acc:>14.1f}% {evolved_acc:>14.1f}% {'🧬' if evolved_acc > human_acc else '👤'}")
print(f"{'Magnitude Error':<25} {human_err:>14.1f}% {evolved_err:>14.1f}% {'🧬' if evolved_err < human_err else '👤'}")
print(f"{'BUY Signal Avg Return':<25} {human_buy:>+14.2f}% {evolved_buy:>+14.2f}% {'🧬' if evolved_buy > human_buy else '👤'}")
print(f"{'SELL Signal Avg Return':<25} {human_sell:>+14.2f}% {evolved_sell:>+14.2f}% {'🧬' if evolved_sell < human_sell else '👤'}")
print(f"{'Fitness Score':<25} {human_fit:>15.1f} {evolved_fit:>15.1f} {'🧬' if evolved_fit > human_fit else '👤'}")

improvement = evolved_acc - human_acc
print(f"\n📈 Direction accuracy improved by {improvement:+.1f} percentage points")

In [ ]:
# === CELL 11: SHOW EVOLVED PARAMETERS ===
print("=" * 70)
print("🧬 EVOLVED OPTIMAL FORECASTER PARAMETERS")
print("=" * 70)

print(f"\n{'Parameter':<25} {'Human':>12} {'Evolved':>12} {'Change':>12}")
print("-" * 65)

for param in FORECAST_PARAMS:
    human_val = human_params[param]
    evolved_val = best_ever.params[param]
    
    if isinstance(human_val, float):
        if human_val != 0:
            change = (evolved_val - human_val) / abs(human_val) * 100
        else:
            change = 0
        print(f"{param:<25} {human_val:>12.3f} {evolved_val:>12.3f} {change:>+11.1f}%")
    else:
        change = evolved_val - human_val
        print(f"{param:<25} {human_val:>12} {evolved_val:>12} {change:>+12}")

# Group by category
print("\n" + "=" * 70)
print("📋 KEY INSIGHTS FROM EVOLUTION:")
print("=" * 70)

# Indicator weights
print("\n🎚️ INDICATOR WEIGHTS (which matter most):")
weights = {k: v for k, v in best_ever.params.items() if k.startswith('w_')}
sorted_weights = sorted(weights.items(), key=lambda x: x[1], reverse=True)
for name, val in sorted_weights:
    indicator = name.replace('w_', '').upper()
    bar = '█' * int(val * 10)
    print(f"   {indicator:<15} {bar:<20} {val:.2f}")

# Thresholds
print("\n🎯 EVOLVED THRESHOLDS:")
print(f"   RSI Oversold: < {best_ever.params['rsi_oversold']:.0f}")
print(f"   RSI Overbought: > {best_ever.params['rsi_overbought']:.0f}")
print(f"   Signal Buy Threshold: {best_ever.params['signal_buy_threshold']:.2f}")
print(f"   Signal Sell Threshold: {best_ever.params['signal_sell_threshold']:.2f}")

# Forecast mechanics
print("\n⚙️ FORECAST MECHANICS:")
print(f"   Daily Bull Drift: {best_ever.params['drift_bull']*100:.2f}%")
print(f"   Daily Bear Drift: {best_ever.params['drift_bear']*100:.2f}%")
print(f"   Max Daily Move: {best_ever.params['max_daily_move']*100:.1f}%")
print(f"   Confidence Decay: {best_ever.params['confidence_decay']:.2f} per day")
print(f"   Decay Start Day: {int(best_ever.params['decay_start_day'])}")

In [ ]:
# === CELL 12: LIVE FORECAST DEMO ===
print("=" * 70)
print("🔮 LIVE FORECAST DEMO (Using Evolved Parameters)")
print("=" * 70)

# Pick top 5 tickers to forecast
demo_tickers = ['RGTI', 'IONQ', 'SMR', 'OKLO', 'TSLA'][:5]

evolved_forecaster = EvolvedForecaster(best_ever)

print(f"\n{'Ticker':<8} {'Price':>10} {'Signal':>8} {'Conf':>8} {'7d Forecast':>12} {'Expected':>10}")
print("-" * 60)

forecasts = []
for ticker in demo_tickers:
    if ticker not in test_features:
        continue
    
    df = test_features[ticker]
    data = test_data[ticker]
    
    row = df.iloc[-1].to_dict()
    current_price = float(data['Close'].iloc[-1])
    volatility = row.get('atr_pct', 2)
    
    signal, confidence, score = evolved_forecaster.generate_signal(row)
    
    # 7-day forecast
    forecast_price = current_price
    for d in range(7):
        forecast_price, _ = evolved_forecaster.forecast_price(
            forecast_price, signal, confidence, d + 1, volatility
        )
    
    expected_return = (forecast_price / current_price - 1) * 100
    
    signal_emoji = '🟢' if signal == 'BUY' else '🔴' if signal == 'SELL' else '🟡'
    
    print(f"{ticker:<8} ${current_price:>9.2f} {signal_emoji} {signal:<6} {confidence*100:>6.0f}% ${forecast_price:>10.2f} {expected_return:>+9.1f}%")
    
    forecasts.append({
        'ticker': ticker,
        'current_price': current_price,
        'signal': signal,
        'confidence': confidence,
        'forecast_7d': forecast_price,
        'expected_return': expected_return
    })

print("\n" + "-" * 60)
buys = [f for f in forecasts if f['signal'] == 'BUY']
sells = [f for f in forecasts if f['signal'] == 'SELL']
print(f"   🟢 BUY signals: {len(buys)}")
print(f"   🔴 SELL signals: {len(sells)}")
print(f"   🟡 HOLD signals: {len(forecasts) - len(buys) - len(sells)}")

In [ ]:
# === CELL 13: SAVE RESULTS ===
results = {
    'generated_at': datetime.now().isoformat(),
    'evolution_config': EVOLUTION_CONFIG,
    'generations_run': EVOLUTION_CONFIG['generations'],
    'human_baseline': {
        'params': human_params,
        'test_direction_accuracy': human_acc,
        'test_magnitude_error': human_err,
        'test_buy_avg': human_buy,
        'test_sell_avg': human_sell,
        'fitness': human_fit
    },
    'evolved_best': {
        'params': best_ever.params,
        'test_direction_accuracy': evolved_acc,
        'test_magnitude_error': evolved_err,
        'test_buy_avg': evolved_buy,
        'test_sell_avg': evolved_sell,
        'fitness': evolved_fit
    },
    'improvement': {
        'direction_accuracy_pct': evolved_acc - human_acc,
        'magnitude_error_reduction': human_err - evolved_err,
        'fitness_improvement': evolved_fit - human_fit
    },
    'evolution_history': history,
    'demo_forecasts': forecasts
}

with open('forecast_evolution_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ Results saved to forecast_evolution_results.json")

try:
    from google.colab import files
    files.download('forecast_evolution_results.json')
    print("📥 Download started!")
except:
    print("(Not in Colab - file saved locally)")

In [ ]:
# === CELL 14: FINAL RECOMMENDATIONS ===
print("=" * 70)
print("📋 FINAL FORECASTER RECOMMENDATIONS")
print("=" * 70)

if evolved_acc > human_acc + 5:  # 5% better
    print("\n🧬 USE EVOLVED FORECASTER!")
    print(f"   Direction accuracy improved by {evolved_acc - human_acc:.1f}%")
    rec = best_ever.params
else:
    print("\n⚖️ BLEND EVOLVED + HUMAN PARAMETERS")
    print("   Evolution found minor improvements.")
    rec = best_ever.params

print("\n🎯 OPTIMAL INDICATOR WEIGHTS:")
print("-" * 50)
for name in ['w_rsi', 'w_macd', 'w_ema_trend', 'w_volume', 'w_momentum', 'w_volatility']:
    indicator = name.replace('w_', '').upper()
    val = rec[name]
    importance = 'HIGH' if val > 1.3 else 'MEDIUM' if val > 0.7 else 'LOW'
    print(f"   {indicator:<15} {val:.2f} ({importance})")

print("\n🔧 COPY THESE SETTINGS TO YOUR FORECASTER:")
print("-" * 50)
print(json.dumps(rec, indent=2))

print("\n" + "=" * 70)
print("✅ FORECAST EVOLUTION COMPLETE!")
print("=" * 70)